Reference:

https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/6%20-%20Transformers%20for%20Sentiment%20Analysis.ipynb

https://github.com/bentrevett/pytorch-seq2seq/blob/master/1%20-%20Sequence%20to%20Sequence%20Learning%20with%20Neural%20Networks.ipynb

Error post:

https://ai.stackexchange.com/questions/26800/pytorch-seq2seq-model-with-pretrained-bert-model-is-throwing-error-during-train



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install Unidecode

     |████████████████████████████████| 245kB 6.8MB/s 


In [3]:
# Imports
import numpy as np
import pandas as pd
import spacy
import random
import re
import math
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
#from torchtext.data import Field,BucketIterator,TabularDataset
from torchtext.legacy.data import Field,BucketIterator,TabularDataset
from sklearn.model_selection import train_test_split

In [4]:
torch.backends.cudnn.deterministic = True

# Data Cleansing

In [5]:
# replacing contraction words

contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am",'i\'m':'i am', "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization'}

In [6]:
# dealing with spl characters
punct = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';',  '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√','∞','θ','÷','α','•','à','−','β','∅','³','π','‘','₹','´','°','™','√²','—–' ]

In [7]:
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }

In [8]:
missplet_dict = {'didnt':'did not',
 "i'm": 'i am',
 'doesnt' : 'does not',
 'isnt' : 'is not',
 'upvote' : 'up vote',
 'wasnt' : 'was not',
 'remindme': 'remind me',
 'lt3' : 'love',
 'upvotes' : 'up votes',
 'shouldnt' : 'should not',
 'hasnt' : 'has not',
 'downvoted' : 'down voted',
 'howd' : ' how do you do',
 'upvoted' : 'up voted',
 'rcasualconversation' : 'casual conversation',
 'cakeday' : 'cake day',
 'downvote' : 'down vote',
 'whatre' : 'what are',
 'contenderwhat' : 'contender what',
 'downvotes' : 'down votes',
 'heshe' : 'he she',
 'tldr' : 'too long did not read',
 'ftfy' : 'Fixed That For You',
 'wbu' : 'what about you',
 'thatd' : 'that would',
 'welp' : 'well',
 'ikr' : 'i know right',
 'realise' : 'realize',
 'thatll' : 'that will',
 'upvoting' : 'up voting',
 'whatd' : ' what did',
 'grey': 'gray',
 'yesno' : 'yes no',
 'flavour' : 'flavor',
 'tooslowly' : 'too slowly',
 'realised' : 'realized',
 'learnt' : 'learn',
 'answers—comment' :  'answers comment',
 'gorillaz' : 'gorilla',
 '“fuck' : 'fuck',
 'accomplishedbeen' : 'accomplished been',
 'whyd' : 'why did',
 'edm' : 'electronic dance music',
 'linkin' : 'linking',
 'hbu' : 'how about you',
 'mustve' : 'must have',
 'whered' : 'where did',
 'continuein' : 'continuing',
 'welldo' : 'well done',
 'downvoter' : 'down voter',
 'friendo' : 'friend',
 '“fixed' : 'fixed',
 'yknow' : 'you know',
 'programme' : 'program',
 'heya' : 'hi',
 'neighbours' : 'neighbor',
 'hmu' : 'hit me up',
 'twinsies' : 'twins',
 '“what' : 'what',
 'inb4' : 'in before',
 'halflife' : 'half life',
 'heyy' : 'hey',
 'summarise' : 'summarize',
 'lpt' : 'life pro tip',
 'gofundme' : 'go find me',
 '5yearold' : '5 year old',
 'noyou' : 'no you',
 'yesyou' : 'yes you',
 'life".' : 'life',
 'yess' : 'yes',
 'greys' : 'gray',
 'daynight' : 'day night',
 'yessss' : 'yes',
 'trickortreating' : 'trick or treat',
 'ninenine' : 'nine nine',
 'dnd' : 'do not disturb',
 'howre' : 'how are',
 'highfive' : 'high five',
 'favour' : 'favor',
 'gtfo' : 'Get the fuck out',
 'necessarythey' : 'necessary they',
 'yessssss' : 'yes',
 'jaja' : 'haha',
 'stopdinner' : 'stop dinner',
 'wristbander' :  'wrist bander',
 'tf2' : 'team fortress',
 'ptsd' : 'post traumatic stress disorder',
 'whataburger' : 'what a burger',
 'youi' : 'you i',
 'nosleep' : 'no sleep',
 'warframe' : 'war frame',
 'what?*' : 'what',
 'yoooo' : 'yes',
 'lul' : 'love you lots',
 'pmed' : 'private messaged',
 ':&gt' : 'get through',
 'pmd' : 'private messaged',
 'ahahah' : 'haha',
 'rsuddenlygay' : 'suddenly gay',
 'rsuicidebywords' : 'suicide by words',
 'rmadlads' :'mad lads',
 '“oh' : 'oh',
 'i‘m' : 'i am',
 'raww' : 'raw',
 'clickbait' : 'click bait',
 'rshowerthoughts' : 'shower thoughts',
 'ymmv' : 'your mileage may vary',
 'imnsho' : 'in my not so humble opinion',
 'shittttt' : 'shit',
 'holllyyyyyyy' : 'holy',
 'foxygen' : 'oxygen',
 'rteenagers' : 'teenagers',
 'gotchu' : 'got you',
 'boredwho' : 'bored who',
 'gt;: ' : 'get through',
 'coldplay' : 'cold play',
 'crosspost' : 'cross post',
 ' #tellyourstoryin5words' : 'tell your story in 5 words',
 'isare' : 'is are',
 'sfw' : 'safe for work',
 'downvoting' :  'down voting',
 'rpointlessstories' : 'pointless stories',
 'now?!' : 'now',
 'apologise' : 'apologize',
 '“they' : 'they',
 'enough’.' : 'enough',
 'goodcool' : 'good cool',
 'gtgt': 'got to go to',
 '‘not' : 'not',
 'upvoter' : 'up voter',
 'breathtakingyour' :  'breath taking our',
 'runexpectedoffice' : 'unexpected office',
 'leavingbi' : 'leaving',
 'rfoundthemobileuser' : 'found the mobile user',
 'thanksif' : 'thanks if',
 'rwholesome' : 'wholesome',
 'lol' : 'laugh out loud',
 'rlounge' : 'lounge',
 'soundcloud' : 'sound cloud',
 'leppard' : 'leopard',
 'rtotallynotrobots' : 'totally not robots',
 'cavetown' : 'cave town',
 'blinddisabled' : 'blind disabled',
 'introverting' : 'introvert',
 'lmaoooo' : 'laughing my ass off',
 'yh' : 'yes',
 'sucks!"america' : 'sucks america',
 'fawlty' : 'faulty',
 'runpopularopinion' : 'run popular opinion',
 'yesssss' : 'yes',
 'crossdress' : 'cross dress',
 'adminsemployees' : 'admins employees',
 'smokedis' : 'smoke',
 'spelt' : 'spell',
 '“far' : 'far',
 'dayweekmonth' : 'day week month',
 'thicc' : 'thick',
 'yaaay' : 'yes',
 'suckswhats' : 'sucks what',
 'durnk' : 'drunk',
 'butterfinger' : 'butter finger',
 'tekken' : 'taken',
 'yayyyy' : 'yes',
 'recall': 'recall',
 'lotr' : 'lord of the rings',
 'mhm' : 'yes',
 'travelled' : 'travel',
 'lolyou' : 'laugh out loud you',
 'rlifeprotips' : 'life pro tips',
 'itwhat' : 'it what',
 'rwallstreetbets' : 'wall street bets',
 'sharptooth' : 'sharp tooth',
 'vsauce' : 'sauce',
 'yayyy' : 'yes',
 'casualconversation' : 'casual conversation',
 'world war 2' : 'world war 2',
 'hiiii' : 'hi',
 'no3' : 'no',
 'selfie' : 'selfie',
 'dreamworks' : 'dream works',
 'bf4' : 'before',
 'hej' : 'hi',
 'flavoured' : 'flavored',
 'daaamn' : 'damn',
 'son' : 'son',
 'you' : 'you',
 'gay".' : 'gay',
 'nword' : 'word',
 'rthedavincicode' : 'the vinci code',
 'brexit' : 'brexit',
 'midtwenties' : 'mid twenties',
 'bingewatch' : 'binge watch',
 'peel”.' : 'peel',
 'yourselfwell' : 'yourself well',
 'yppah' : 'yes',
 'rbisexual' : 'bisexual',
 'kindda' : 'kind of',
 'runderratedcomments' : 'underrated comments',
 'backmaybe' : 'back may be',
 'itbut' : 'it but',
 'suggestionsi' : 'suggestions',
 'motherfuckerglad' : 'mother fucker glad',
 'nvm' : 'never mind',
 'songshelp' : 'songs help',
 'positivityfun' : 'positivity fun',
 'indoorone' : 'indore one',
 'timehow' : 'time how',
 'lolbut' : 'laugh out loud but',
 'manwoman' : 'man woman',
 'feelingdoing' : 'feeling doing',
 'friendscould' : 'friends cloud',
 'rhappyrelationships' : 'happy relationships',
 'do. Lonely' : 'do lonely',
 'realisation' : 'realization',
 'woooow' : 'wow',
 'rcatsstandingup' : 'cats standing up',
 'goldenboye' : 'golden boy',
 'weeki' : 'week',
 'numbah' : 'number',
 'yeahh' : 'yes',
 'youim' : 'you i am',
 'toowhat' : 'too what',
 'deffinetly' : 'definitely',
 'game3' : 'game',
 'xpost' : 'post',
 'rchildfree' : 'child free',
 'music2' : 'music',
 'quotebook' : 'quote book',
 'ufuckswithducks' : 'fuck with duck',
 'lold' : 'old',
 'whaaaaat' : 'what',
 'flavours' : 'flavors',
 'rpics' : 'pictures',
 'piano2' : 'piano',
 'requirementsi' : 'requirements',
 'himher' : 'him her',
 'yeaaah' : 'yes',
 'rsuicidewatch' : 'suicide watch',
 'damnnnn' : 'damn',
 'longsword' : 'long word',
 'awfuli' : 'awefully',
 'humour' : 'humor',
 'fooddrink' : 'food drink',
 'hby' : 'how about you',
 'roomwhat' : 'room what',
 'selfies' : 'selfie',
 'feelsbadman' : 'feels bad man',
 'youuu' : 'you',
 'mustnt' : 'must not',
 'nederlands' : 'netherlands',
 'aaaaand' : 'and',
 'smoll' : 'small',
 'yus' : 'yes',
 'madlad' : 'mad lad',
 'rniceguys' : 'nice guys',
 'uyourearealcunt' : ' you are a real cunt',
 'rimsorryjon' : 'i am sorry jon',
 'yet."' : 'yet',
 'edit😬' : 'edit',
 'retreival' : 'retrieval',
 'haaaaate' : 'hate',
 'musicianband' : 'musician band',
 'wingies' : 'wings',
 'money4' : 'money',
 'work2' : 'work',
 'goodi' : 'good',
 'touchstarved' : 'touch starved',
 'grindr' : 'grinder',
 'rblunderyears' : 'blunder years',
 'rcursedcomments' : 'cursed comments',
 '‘em' : 'them',
 'rstaticsnake' : 'static snake',
 'marcy' : 'mercy',
 'mightve' : 'might have',
 'cuteits' : 'cute',
 'whatim' : 'what i am',
 'havingfun' : 'having fun',
 'exwifes' : 'ex wives',
 'noyoure' : 'no you are',
 'aaaaand' : 'and',
 'smoll' : 'small',
 'yus' : 'yes',
 "i'hv": "i have",
 "i'll": "i will",
 '☺️':'😊',
 'ampx###b': 'amp bitch',"i've": 'i have','rwholesomememes':'wholesome memes',
 'hello😀😊☺🙌👍':'hello 😀 😊','͜ʖ': '😊', '😂😂':'😂','😂😂😂':'😂','ayyyy':'yes','\U0001f970':'😊', '🅱️ailure' : 'big failure', "goin'": 'going','👉😎👉': '😎', "d'aww" : 'so cute','ulionghost': ' you lion ghost',"fuckin'": 'fucking', 'uiwinalot7': ' you i win a lot', 'goodra': 'good',
'pie5': 'being soft', '😀😊☺': '😀', 'uwaterguy##': ' you water guy', ' rseriousconversation': 'serious conversation','runexpectedfactorial':'unexpected factorial', "i'mma": 'i am',
'👏👏':'👏', "lol'd": "laughed out loud", 'ಠಠ': '😎', 'rtea': 'tea', '🤣🤣🤣': '🤣','👏👏👏':'👏',"you'": 'you', "d'aw": 'so cute', 'pmmeyogurtpics':'send me yogurt pictures',
"sirma'am": 'sir madam','rusernamechecksout': 'user name checks out', 'lt3lt3lt3': 'love', 'bookmovie': 'book movie', '😀😊☺🤗😇': '😀 😊 🤗 😇',
'rpopping': 'popping','gt##': 'got to go', 'rwoooosh': 'woooosh', 'rr4r': 'redditor for redditor','ustaticsnake': 'you static snake','eli5':'explain in simple words',
'mrbeast': 'beast', 'rkarmaroulette':'karma roulette','😭😭':'😭','️\U0001f9e1💛💚💙💜':'love','🤔🤔🤔':'🤔','rgatekeeping':'gate keeping', 'moviestv': 'movie television','dinozo':'dinosaur', 'scarn': 'mayhem',
'\U0001f9e1💛💚💙💜': 'love', "pm'd":'messaged','ayyye':'yes', '\U0001f92a':'😀', 'gtno': 'get the fuck out', "walkin'": 'walking', 'rknightsofpineapple':'knights of pineapple',
'##what': 'what','rbrandnewsentence':'brand new sentence', 'bhosadiwale':'son of a bitch', 'runexpectedthanos': 'unexpected thanos',"'murica": 'america',
'udonaldduck':'you donald duck','rjobs':'jobs' ,'uitsthesnake': 'it is the snake','💕💕': '💕'}

Pre-processing functions

In [9]:
def clean_contractions(text, mapping):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([mapping[t] if t in mapping else mapping[t.lower()] if t.lower() in mapping else t for t in text.split(" ")])
    return text

In [10]:
def clean_special_chars(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, '')
    
    specials = {'\u200b': ' ', '…': ' ', '\ufeff': ''}  # to be updated again upon checking the coverage
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text

In [11]:
def remove_newlines(sent):
  sent = re.sub(r'\s+', " ", sent )
  return sent

In [12]:
def clean_numbers(x):
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
        #x = re.sub('[0-9]', '#', x)
    return x

In [13]:
def clean_missplets(text, mapping):
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text

Data Loading

In [14]:
# fetching data from csv file to dataframe
reddit = pd.read_csv('/content/drive/My Drive/Data/Conversations/casual_data_windows.csv')

In [15]:
reddit.head(2)

,Unnamed: 0,0,1,2
0,0,What kind of phone(s) do you guys have?,I have a pixel. It's pretty great. Much better...,Does it really charge all the way in 15 min?
1,1,I have a pixel. It's pretty great. Much better...,Does it really charge all the way in 15 min?,"Pretty fast. I've never timed it, but it's und..."


In [16]:
# creating an empty dataframe to hold consecutive conversation-pairs
df_conv = pd.DataFrame(columns=['sent1', 'sent2'])

In [17]:
# filling the dataframe with conversation pairs from reddit dataframe
for index, row in reddit.iterrows():
    df_conv = df_conv.append({'sent1':row["0"],'sent2':row["1"]},ignore_index=True)
    df_conv = df_conv.append({'sent1':row["1"],'sent2':row["2"]},ignore_index=True)

Data cleansing activities

In [18]:
df_conv['sent1'] = df_conv['sent1'].apply(lambda x: clean_contractions(x, contraction_mapping))
df_conv['sent2'] = df_conv['sent2'].apply(lambda x: clean_contractions(x, contraction_mapping))

In [19]:
df_conv['sent1'] = df_conv['sent1'].apply(lambda x: clean_special_chars(x, punct, punct_mapping))
df_conv['sent2'] = df_conv['sent2'].apply(lambda x: clean_special_chars(x, punct, punct_mapping))

In [20]:
df_conv['sent1'] = df_conv['sent1'].apply(lambda x: clean_contractions(x, contraction_mapping))
df_conv['sent2'] = df_conv['sent2'].apply(lambda x: clean_contractions(x, contraction_mapping))

In [21]:
df_conv['sent1'] = df_conv['sent1'].apply(lambda x : x.lower())
df_conv['sent2'] = df_conv['sent2'].apply(lambda x : x.lower())

In [22]:
df_conv['sent1'] = df_conv['sent1'].apply(lambda x : remove_newlines(x))
df_conv['sent2'] = df_conv['sent2'].apply(lambda x : remove_newlines(x))


In [23]:
df_conv['sent1'] = df_conv['sent1'].apply(lambda x: clean_numbers(x))
df_conv['sent2'] = df_conv['sent2'].apply(lambda x: clean_numbers(x))

In [24]:
df_conv['sent1'] = df_conv['sent1'].apply(lambda x: clean_numbers(x))
df_conv['sent2'] = df_conv['sent2'].apply(lambda x: clean_numbers(x))

In [25]:
df_conv['sent1'] = df_conv['sent1'].apply(lambda x: clean_missplets(x, missplet_dict))
df_conv['sent2'] = df_conv['sent2'].apply(lambda x: clean_missplets(x, missplet_dict))

In [26]:
print(df_conv.shape)
df_conv.head()

(112594, 2)


,sent1,sent2
0,what kind of phones do you guys have,i have a pixel it is pretty great much better ...
1,i have a pixel it is pretty great much better ...,does it really charge all the way in ## min
2,i have a pixel it is pretty great much better ...,does it really charge all the way in ## min
3,does it really charge all the way in ## min,pretty fast i have never timed it but it is un...
4,does it really charge all the way in ## min,pretty fast i have never timed it but it is un...


In [27]:
duplicateRowsDF = df_conv[df_conv.duplicated()]

In [28]:
# removing duplicate rows
df_conv = df_conv.drop_duplicates(subset=None, keep='first', inplace=False)

In [29]:
print(df_conv.shape)
df_conv.head()

(71052, 2)


,sent1,sent2
0,what kind of phones do you guys have,i have a pixel it is pretty great much better ...
1,i have a pixel it is pretty great much better ...,does it really charge all the way in ## min
3,does it really charge all the way in ## min,pretty fast i have never timed it but it is un...
5,pretty fast i have never timed it but it is un...,cool i have been thinking of getting one my ph...
6,what kind of phones do you guys have,samsung galaxy j1 it is my first cell phone an...


Word to index dictionary creation for our dataset

In [30]:
# filtering out unique sentence set from conversation pairs. this is needed to be done before creating word to index dictionary

sent1 = df_conv['sent1']
sent2 = df_conv['sent2']

In [31]:
import torch

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)


In [32]:
!pip install transformers

     |████████████████████████████████| 1.9MB 6.1MB/s 
     |████████████████████████████████| 3.2MB 24.8MB/s 
     |████████████████████████████████| 890kB 54.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=a0e795ab8497ce6c7f73fb7c4dc3183bb1d968c8d1466f0659d25c68ebe70b52
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [33]:
import transformers
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [34]:
len(tokenizer.vocab)

30522

In [35]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

[CLS] [SEP] [PAD] [UNK]


In [36]:
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


In [37]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']

print(max_input_length)

512


In [38]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

In [39]:
from torchtext.legacy import data

sent1 = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

sent2 = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

In [40]:
train_data , valid_data = train_test_split(df_conv,test_size = 0.2)

In [41]:
%cd '/content/drive/My Drive/Data/Conversations/'

%pwd

/content/drive/My Drive/Data/Conversations


'/content/drive/My Drive/Data/Conversations'

In [42]:
train_data.to_csv('train.csv',index = False)
valid_data.to_csv('test.csv',index = False)

In [43]:
data_fields = [('sent1',sent1),('sent2',sent2)]

In [44]:
# TabularDataset(Dataset):Defines a Dataset of columns stored in CSV, TSV, or JSON format.
# Create a TabularDataset given a path, file format, and field list

train_data , valid_data = TabularDataset.splits(path ='/content/drive/My Drive/Data/Conversations/' ,train='train.csv', test ='test.csv', format='csv', fields=data_fields)

In [45]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of testing examples: {len(valid_data)}")

Number of training examples: 56842
Number of testing examples: 14212


In [46]:
print(vars(train_data.examples[6]))

{'sent1': [2073, 2515, 3071, 2444, 1045, 2444, 1999, 5862, 11333], 'sent2': [2141, 1999, 14700, 4058, 2444, 1999, 8770, 4058, 2005, 1996, 2293, 1997, 2643, 2131, 2033, 2041, 1997, 4058]}


In [47]:
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[6])['sent1'])

print('Sent1 tokens:',tokens)

tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[6])['sent2'])

print('Sent2 tokens:',tokens)

Sent1 tokens: ['where', 'does', 'everyone', 'live', 'i', 'live', 'in', 'seattle', 'wa']
Sent2 tokens: ['born', 'in', 'dayton', 'ohio', 'live', 'in', 'canton', 'ohio', 'for', 'the', 'love', 'of', 'god', 'get', 'me', 'out', 'of', 'ohio']


In [48]:
# building vocabulary for the responses(sent2)
sent2.build_vocab(train_data)

In [49]:
len(sent2.vocab)

15183

In [50]:
from torchtext.legacy.data import BucketIterator,TabularDataset

BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_data, valid_data), 
    batch_size = BATCH_SIZE, 
    device = device)

In [51]:
from transformers import BertTokenizer, BertModel

bert = BertModel.from_pretrained('bert-base-uncased')

In [52]:
# Encoder

class Encoder(nn.Module):
    def __init__(self, bert, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.bert = bert
        
        emb_dim = bert.config.to_dict()['hidden_size']
              
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, batch_first = True,dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, sent1):
        
        #sent1 = [sent1 len, batch size]
        
        with torch.no_grad():
            embedded = self.bert(sent1)[0]
        
        #embedded = [sent1 len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [sent1 len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

In [53]:
# Decoder

class Decoder(nn.Module):
    def __init__(self, bert, hid_dim, output_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.bert = bert        
        emb_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, batch_first = True, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)


    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        with torch.no_grad():
            embedded = self.bert(sent2)[0]
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

In [54]:
# Seq2seq

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, sent1, sent2, teacher_forcing_ratio = 0.5):
        
        #sent1 = [sent1 len, batch size]
        #sent2 = [sent2 len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = sent2.shape[1]
        sent2_len = sent2.shape[0]
        sent2_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(sent2_len, batch_size, sent2_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(sent1)
        
        #first input to the decoder is the <sos> tokens
        input = sent2[0,:]
        
        for t in range(1, sent2_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = sent2[t] if teacher_force else top1
        
        return outputs



In [55]:
OUTPUT_DIM = len(sent2.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(bert, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(bert, HID_DIM, OUTPUT_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [56]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (LayerN

In [57]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 126,724,687 trainable parameters


In [58]:
optimizer = optim.Adam(model.parameters())

In [59]:
SENT2_PAD_IDX = pad_token_idx

criterion = nn.CrossEntropyLoss(ignore_index = SENT2_PAD_IDX)

In [60]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        sent1 = batch.sent1
        sent2 = batch.sent2
        
        optimizer.zero_grad()
        
        output = model(sent1, sent2)
        
        #sent2 = [sent2 len, batch size]
        #output = [sent2 len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        sent2 = sent2[1:].view(-1)
        
        #sent2 = [(sent2 len - 1) * batch size]
        #output = [(sent2 len - 1) * batch size, output dim]
        
        loss = criterion(output, sent2)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)


In [61]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            sent1 = batch.sent1
            sent2 = batch.sent2

            output = model(sent1, sent2, 0) #turn off teacher forcing

            #sent2 = [sent2 len, batch size]
            #output = [sent2 len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            sent2 = sent2[1:].view(-1)

            #sent2 = [(sent2 len - 1) * batch size]
            #output = [(sent2 len - 1) * batch size, output dim]

            loss = criterion(output, sent2)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [62]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [63]:
N_EPOCHS = 1
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')


AttributeError: ignored